# Convert climate datasets for ML algorithms

Here, we will convert the CliMT outputs to a format for ML training. Also we will create normalization files for neural network training.

In [1]:
from functions import *
%matplotlib inline

In [2]:
input_vars = ['air_temperature', 'specific_humidity', 'eastward_wind', 'northward_wind', 
              'air_pressure']
output_vars = [
    'air_temperature_tendency_from_EmanuelConvection', 
    'specific_humidity_tendency_from_EmanuelConvection', 
    'eastward_wind_tendency_from_EmanuelConvection', 
    'northward_wind_tendency_from_EmanuelConvection',
    'convective_precipitation_rate'
]

In [12]:
time_slice=slice(-365*4, None)
inputs = xr.open_mfdataset(
    'inputs_and_kua.nc', chunks={'time': 10}, combine='by_coords'
)[input_vars].transpose('time', 'mid_levels', 'lat', 'lon').isel(time=time_slice)
outputs = xr.open_mfdataset(
    'outputs_and_kua.nc', chunks={'time': 10}, combine='by_coords'
)[output_vars].transpose('time', 'mid_levels', 'lat', 'lon').isel(time=time_slice)

## Stack data

[sample, stacked_levels]

In [7]:
#EXPORT
def convert_data(raw_input_fn, raw_output_fn, conv_input_fn, conv_output_fn,
                 input_vars=input_vars, output_vars=output_vars, time_slice=slice(0, None)):
    """Convert raw climate model outputs to stacked arrays for neural network training"""
    inputs = xr.open_mfdataset(
        raw_input_fn, chunks={'time': 10}, combine='by_coords'
    )[input_vars].transpose('time', 'mid_levels', 'lat', 'lon').isel(time=time_slice)
    outputs = xr.open_mfdataset(
        raw_output_fn, chunks={'time': 10}, combine='by_coords'
    )[output_vars].transpose('time', 'mid_levels', 'lat', 'lon').isel(time=time_slice)
    
    inputs = xr.concat(
        [inputs[v] for v in input_vars], dim='mid_levels'
    ).rename('inputs')
    outputs = xr.concat(
        [outputs[v] for v in output_vars], dim='mid_levels'
    ).rename('outputs')
    
    inputs = inputs.stack(
        sample=('time', 'lat', 'lon')
    ).transpose().reset_index('sample')
    outputs = outputs.stack(
        sample=('time', 'lat', 'lon')
    ).transpose().reset_index('sample')
    
    inputs.to_netcdf(conv_input_fn)
    outputs.to_netcdf(conv_output_fn)
    
    return inputs, outputs

In [11]:
convert_data(
    'inputs_and_kua.nc', 'outputs_and_kua.nc',
    'stacked_inputs_and_kua.nc', 'stacked_outputs_and_kua.nc',
    time_slice=slice(-365*4, None)
)

(<xarray.DataArray 'inputs' (sample: 2990080, mid_levels: 50)>
 dask.array<shape=(2990080, 50), dtype=float64, chunksize=(10240, 10)>
 Coordinates:
     time     (sample) datetime64[ns] 2000-12-31T00:30:00 ... 2004-12-29T00:30:00
     lat      (sample) int64 0 0 0 0 0 0 0 0 0 0 ... 31 31 31 31 31 31 31 31 31
     lon      (sample) int64 0 1 2 3 4 5 6 7 8 9 ... 55 56 57 58 59 60 61 62 63
 Dimensions without coordinates: sample, mid_levels,
 <xarray.DataArray 'outputs' (sample: 2990080, mid_levels: 41)>
 dask.array<shape=(2990080, 41), dtype=float64, chunksize=(10240, 10)>
 Coordinates:
     time     (sample) datetime64[ns] 2000-12-31T01:00:00 ... 2004-12-29T01:00:00
     lat      (sample) int64 0 0 0 0 0 0 0 0 0 0 ... 31 31 31 31 31 31 31 31 31
     lon      (sample) int64 0 1 2 3 4 5 6 7 8 9 ... 55 56 57 58 59 60 61 62 63
 Dimensions without coordinates: sample, mid_levels)

## Compute normalization files

To avoid weird effect from dividing by small numbers, I will compute the std for each variable over all levels and use this.

In [13]:
#EXPORT
def compute_means_stds(ds, sampling_interval=100):
    means = {v: ds[v].isel(time=slice(0, None, sampling_interval)).mean().values 
             for v in ds} 
    stds = {v: ds[v].isel(time=slice(0, None, sampling_interval)).std().values 
             for v in ds} 
    return means, stds

In [14]:
def broadcast_norm(ds, stat, var):
    arr = []
    for v in var:
        arr += [stat[v]] * (len(ds[v].mid_levels) if hasattr(ds[v], 'mid_levels') else 1)
    return np.array(arr)

In [15]:
input_means, input_stds = [broadcast_norm(inputs, stat, input_vars) 
                           for stat in compute_means_stds(inputs)]

In [16]:
output_means, output_stds = [broadcast_norm(outputs, stat, output_vars) 
                           for stat in compute_means_stds(outputs)]

In [17]:
with open('norm_arrs_and_kua.pkl', 'wb') as f:
    pickle.dump((input_means, input_stds, output_means, output_stds), f)